In [ ]:
import numpy as np
import scipy.stats as si
from statsmodels.tsa.seasonal import STL
import yfinance as yf
from datetime import datetime
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cnf = {
    "start": "2020-01-01",
    "end": "2025-01-01",
    "ticker": "NVDA",
    "trend": None,
    "stl_period": 21,
    "robust": True,
    "market_ticker": "^GSPC",
    "rfr": 0.04
}

In [ ]:
def get_data(ticker, start, end):
  data = yf.Ticker(cnf["ticker"]).history(start=cnf["start"], end=cnf["end"])
  return data.drop(["Dividends", "Stock Splits"], axis=1)

In [ ]:
data = get_data(ticker=cnf["ticker"], start=cnf["start"], end=cnf["end"])
market = get_data(ticker=cnf["market_ticker"], start=cnf["start"], end=cnf["end"])

In [ ]:
def apply_decomposition_metrics(log_returns: pd.DataFrame|np.ndarray, 
                            period: int|None=None, trend: int|None=None,  
                            robust: bool=True) -> pd.DataFrame:
  if not isinstance(log_returns, pd.DataFrame) and period is None:
    raise ValueError("If log_returns is not DataFrame, period should be specified")

  stl = STL(log_returns[1:,], period=period, robust=robust, trend=trend)
  res = stl.fit()
  
  return res.seasonal, res.trend

def compute_realized_vol(data: pd.DataFrame, lambda_: int|None=None, window: int|None=21):
  data = data.copy()
  data["LogRet"] = np.log(data['Close']).diff()
  if lambda_:
    data["RealizedVol"] = data["LogRet"].emw((1-lambda_)).std() * np.sqrt(252)

  elif window:
    data["RealizedVol"] = data["LogRet"].rolling(window).std() * np.sqrt(252)

  return data

def apply_greeks(data):
  pass


In [ ]:
market = compute_realized_vol(market)
data = compute_realized_vol(data)
data["Season"], data["Trend"]  = apply_decomposition_metrics(data["LogRet"], period=cnf["stl_period"])
